# Usage of the Imagine Model library

In this tutorial, we focus on introducing the model library and the IMAGINE built-in simulators as a building block of the IMAGINE package and how to use it for assembling a Bayesian analysis pipeline.
     

## 1) Initialize the models



In [19]:
import imagine as img
import imagine.observables as img_obs
import numpy as np
import astropy.units as u

In [20]:
# Fixes numpy seed to ensure this notebook leads always to the same results
np.random.seed(42)

# A cartesian grid can be constructed as follows
cartesian_grid = img.fields.UniformGrid(box=[[-15*u.kpc, 15*u.kpc],
                                             [-15*u.kpc, 15*u.kpc],
                                             [-15*u.kpc, 15*u.kpc]],
                                        resolution = [15,15,15])


In [21]:
jf_12_field = img.fields.library.JF12Regular(grid=cartesian_grid) # using default parameters

In [22]:
jf_12_factory = img.fields.FieldFactory(
    field_class=jf_12_field,
    grid=cartesian_grid,
    active_parameters=[],
    default_parameters = {})
        

In [23]:
ymw16_field = img.fields.library.YMW16(grid=cartesian_grid) # using default parameters

In [24]:
ymw16_factory = img.fields.FieldFactory(
    field_class=ymw16_field,
    grid=cartesian_grid,
    active_parameters=[],
    default_parameters = {'central_density': 1*u.cm**-3,
                          'scale_radius': 3.0*u.kpc,
                          'scale_height': 0.5*u.kpc})

In [25]:
nside=2
size = 12*nside**2 

# Generates the container for mock dataset
fd_dset = img_obs.FaradayDepthHEALPixDataset(data=np.empty(size)*u.rad/u.m**2)

# Appends them to an Observables Dictionary
trigger = img_obs.Measurements(fd_dset)

# Prepares the Hammurabi simmulator for the mock generation
mock_generator = img.simulators.RMSimulator(measurements=trigger)

AssertionError: No valid observable was requested!

In [ ]:
pipeline = img.pipelines.MultinestPipeline(run_directory='../runs/tutorial_example/',
                                           simulator=simulator, 
                                           show_progress_reports=True,
                                           factory_list=factory_list, 
                                           likelihood=likelihood, 
                                           ensemble_size=1, n_evals_report=15)
pipeline.sampling_controllers = {'n_live_points': 500}